---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

#print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# write data from tsv to pandas dataframes
train = pd.read_csv(train_file_path, sep = "\t", names = ["h/s","text"])
test = pd.read_csv(test_file_path, sep = "\t", names = ["h/s","text"])


In [ ]:
# look and explore data
train.info()
print()
test.info()

In [ ]:
train.head(10)

In [ ]:
train["len"] = train["text"].apply(len)
test["len"] = test["text"].apply(len)

In [ ]:
sns.distplot(train['len'], kde=False)

In [ ]:
sns.distplot(test['len'], kde=False)

In [ ]:
print(train["len"].median(),
      train["len"].mean())

In [ ]:
sns.countplot(test['h/s'])
plt.xlabel("sms")
plt.title("Num of h/s")

In [ ]:
# make labels
test_data, train_data = test["text"], train["text"]
lab_enc = LabelEncoder()
test_labels = lab_enc.fit_transform(test["h/s"]).reshape(-1,1) 
train_labels = lab_enc.fit_transform(train["h/s"]).reshape(-1,1)
# 0 - ham, 1 - spam

In [ ]:
train_data

In [ ]:
VOCAB_SIZE = 3000
MAXLEN = 200

# creat token.
# make tokens for most popular words (count tokens VOCAB_SIZE-1)
tok = Tokenizer(num_words=VOCAB_SIZE)

# updating tokens dictionary on base of texts list
tok.fit_on_texts(train_data)

# converting each text in texts to a sequence of integers.
train_data = tok.texts_to_sequences(train_data)
test_data = tok.texts_to_sequences(test_data)


In [ ]:

# if the sms is greater than MAXLEN words then trim off the extra words
# if the sms is less than MAXLEN words add the necessary amount of 0's 
# to make it equal to MAXLEN.
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [ ]:
train_data[0]

In [ ]:
# creating a model
model = tf.keras.Sequential([
      tf.keras.layers.Embedding(VOCAB_SIZE, 64),
      tf.keras.layers.LSTM(64),
      tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

In [ ]:
#compile the model
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])

In [ ]:
# teach the model
history = model.fit(train_data, train_labels, batch_size=64, epochs=7, validation_split=0.2)

In [ ]:
# check the model in test data
results = model.evaluate(test_data, test_labels)
print(f"Loss: {results[0]}, Acc: {results[1]}")

In [ ]:
# try our model in some text
def predict_message(pred_text):
  """ 
     function to predict messages based on model
     (should return list containing prediction and label,
     ex. [0.008318834938108921, 'ham'])
  """
  tokens = tok.texts_to_sequences([pred_text])
  encode_text = sequence.pad_sequences(tokens, MAXLEN)
  pred = np.zeros((1,200))
  pred[0] = encode_text
  result = model.predict(pred) 
  f = lambda x: "spam" if round(x + 0.01) else "ham"
  prediction = [result[0][0], f(result[0][0])]
 
  return (prediction)

pred_text = "sale today! to stop texts call 98912460324"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
